In [1]:
# 필요한 라이브러리 설치
!pip install faiss-cpu
!pip install sentence-transformers
!pip install transformers
!pip install fastapi uvicorn nest_asyncio pyngrok

In [2]:
# ================================
# 1. Sentence-BERT 임베딩
# ================================
from fastapi import FastAPI, Request
from pydantic import BaseModel
from typing import List
import numpy as np
import faiss

from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

app = FastAPI()

# Sentence-BERT 모델 로드
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # 문장 임베딩 특화

# GPT-Neo 모델 로드
gpt_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
gpt_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token
qa_generator = pipeline("text-generation", model=gpt_model, tokenizer=gpt_tokenizer)

def get_bert_embedding(text):
    """
    Sentence-BERT 기반 임베딩 생성
    """
    embedding = embedding_model.encode([text], normalize_embeddings=True)  # 코사인 유사도용 정규화 포함
    return embedding.astype('float32')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [3]:
# ================================
# 2. FAISS 인덱스 생성
# ================================
def create_faiss_index(embeddings):
    """
    FAISS 인덱스 생성 (코사인 유사도용 Inner Product)
    """
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index

In [4]:
# ================================
# 3. GPT-Neo 응답 생성 (Pipeline 사용)
# ================================
# GPT-Neo 로드
gpt_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
gpt_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token  # pad token 설정

# Pipeline 구성
qa_generator = pipeline("text-generation", model=gpt_model, tokenizer=gpt_tokenizer)

def generate_gpt_response(context, question):
    """
    GPT-Neo를 사용한 답변 생성
    """
    prompt = f"문맥: {context}\n질문: {question}\n답변:"
    response = qa_generator(
        prompt,
        max_length=500,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        num_return_sequences=1,
        pad_token_id=gpt_tokenizer.eos_token_id
        )
    answer = response[0]['generated_text'].split("답변:")[-1].strip()
    return answer

Device set to use cuda:0


In [5]:
# Q&A 데이터셋
qa_texts = [
    "AI stands for Artificial Intelligence, which refers to technologies that enable machines to learn and reason like humans.",
    "FAISS is a high-dimensional vector search library developed by Facebook AI.",
    "BERT is a pre-trained transformer model designed for natural language understanding.",
    "GPT-Neo is a transformer-based language generation model trained on large-scale data.",
    "Cosine similarity is a metric that measures how similar two vectors are based on their direction."
]

embeddings = np.vstack([get_bert_embedding(text) for text in qa_texts])
index = create_faiss_index(embeddings)

# --------------------------
# 입력/출력 모델 정의
# --------------------------
class QARequest(BaseModel):
    question: str
    top_k: int = 3

class QAResponse(BaseModel):
    question: str
    response: str
    context: List[str]

# --------------------------
# FastAPI 라우터
# --------------------------
@app.post("/qa", response_model=QAResponse)
def qa_endpoint(request: QARequest):
    question = request.question
    top_k = request.top_k

    # 질문 임베딩
    query_vec = get_bert_embedding(question)
    distances, indices = index.search(query_vec, top_k)

    # 유사 문맥 추출
    similar_texts = [qa_texts[i] for i in indices[0]]
    context = " ".join(similar_texts)

    # GPT 응답 생성
    gpt_response = generate_gpt_response(context, question)

    return QAResponse(
        question=question,
        response=gpt_response,
        context=similar_texts
    )

In [6]:
# 2. ngrok 실행 코드
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# 3. ngrok 설정
ngrok.set_auth_token("2v1Fi5CEzLumREBpheNMIIepRlM_7uLFbq5PGe81hmEZiAe9K")  # 🔑 Ngrok 토큰 입력 (한 번만 필요)
ngrok.kill()  # 이전 터널 종료
public_url = ngrok.connect(3000)  # 로컬 3000 포트를 외부에 노출
print("🔗 Public URL:", public_url.public_url)

# 4. 이벤트 루프 충돌 방지 (Colab 전용)
nest_asyncio.apply()

# 5. uvicorn 실행
uvicorn.run(app, host="0.0.0.0", port=3000)

🔗 Public URL: https://006c-34-82-40-125.ngrok-free.app


INFO:     Started server process [6809]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:3000 (Press CTRL+C to quit)


INFO:     155.230.85.158:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     155.230.85.158:0 - "GET /openapi.json HTTP/1.1" 200 OK


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


INFO:     155.230.85.158:0 - "POST /qa HTTP/1.1" 200 OK
INFO:     155.230.85.158:0 - "POST /qa HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [6809]


In [ ]:
# ================================
# 4. Q&A 파이프라인
# ================================
def qa_pipeline(question, index, qa_texts, k=3):
    """
    Q&A 시스템 전체 파이프라인
    """
    # 1. 질문 임베딩
    query_vec = get_bert_embedding(question)

    # 2. FAISS 검색
    distances, indices = index.search(query_vec, k)

    # 3. 상위 k개의 유사 문장 추출
    similar_texts = [qa_texts[i] for i in indices[0]]

    # 4. 문맥 조합
    context = " ".join(similar_texts)

    # 5. GPT로 응답 생성
    gpt_response = generate_gpt_response(context, question)

    # 🔍 로그 출력
    print("📌 질문:", question)
    print("🔎 검색된 문장:")
    for i, text in enumerate(similar_texts, 1):
        print(f"{i}. {text}")
    print("🧠 GPT 응답:", gpt_response)

    return gpt_response

In [ ]:
# ================================
# 5. 실행 테스트
# ================================
# Q&A 데이터셋
qa_texts = [
    "AI는 인공지능의 줄임말로, 기계가 사람처럼 학습하고 추론할 수 있도록 하는 기술입니다.",
    "FAISS는 Facebook AI에서 만든 고차원 벡터 검색 라이브러리입니다.",
    "BERT는 자연어 이해를 위한 사전 훈련된 트랜스포머 모델입니다.",
    "GPT-neo는 언어 생성을 위한 트랜스포머 모델로, 대규모 데이터를 바탕으로 훈련되었습니다.",
    "코사인 유사도는 두 벡터의 방향성을 비교해 얼마나 유사한지 측정하는 지표입니다."
]

# 문장 임베딩 벡터 생성
embeddings = np.vstack([get_bert_embedding(text) for text in qa_texts])

# FAISS 인덱스 생성
index = create_faiss_index(embeddings)

# 테스트 질문
question = "인공지능이 뭐야?"

# 응답 생성
response = qa_pipeline(question, index, qa_texts)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


📌 질문: 인공지능이 뭐야?
🔎 검색된 문장:
1. AI는 인공지능의 줄임말로, 기계가 사람처럼 학습하고 추론할 수 있도록 하는 기술입니다.
2. 코사인 유사도는 두 벡터의 방향성을 비교해 얼마나 유사한지 측정하는 지표입니다.
3. BERT는 자연어 이해를 위한 사전 훈련된 트랜스포머 모델입니다.
🧠 GPT 응답: 전투에 사물이 취약점에 따라 있다. 유사도 전투를 통해 해석한 반응한 훈련을 통해 그 전과 연기와 연기를 시작한 반응훈련을 써도 연결하여 이어야 합니다.
“전과 연�
response:  전투에 사물이 취약점에 따라 있다. 유사도 전투를 통해 해석한 반응한 훈련을 통해 그 전과 연기와 연기를 시작한 반응훈련을 써도 연결하여 이어야 합니다.
“전과 연�
